# Assignment 1: Classifying handwritten digits using probability theory

# 1. Introduce the problem in your own words. You should mention what data we use, what we want to do with it and how we will do it. Explain with your own words the Naive Bayes classifier, it's assumptions and how this can classify the digits $0-9$.

Write your answer here.

The data set cointains images with a black background and drawings of numbers from zero to nine, in white.  The objective is to build a classifier that can correctly distinguish what is the number drawn in the image.  For this, the Naive Bayes classifier will be used. This is an aproximation where each pixel is analysed indpendently of the others.  The advantage of this, is that the number of calclations required is significantly smaller, rather than analysing whole groups of pixels within the image.  To use the Naive Bayes classifier, every pixel is assumed to be independent of each other.  In practice, this may not be the case, but by using the Naive Bayes we only care about a broad degree of accuracy.  

By combining Naive Bayes with a classification rule, we can attempt to implement this.

In [2]:
from __future__ import division
import numpy as np
import pylab as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import plot_confusion_matrix


ModuleNotFoundError: No module named 'pylab'

# 2. Load the data

In [1]:
# load data
data = np.load("mnist_bin.npz") # insert your answer here
## If running into further errors, consult "np.load("mnist_bin.npz",'',allow_pickle=False)"

# get vector representation of binary digits
X = data['X']

# get binary labels
y = data['y']

print('The shape of X is (%d, %d)' % X.shape)
print('The shape of y is (%d)\n' % y.shape)

# Dimensions
N, D = X.shape

print('Number of images: %d' % N)
print('Number of pixels: %d' % D)

NameError: name 'np' is not defined

#### Run the code beneath. It plots 10 images of each digit. 


In [ ]:
# Graphical representation function, don't worry about it
def show_image(x, title="", clim=None, cmap=plt.cm.gray, colorbar=False):
    ax = plt.gca()
    im = ax.imshow(x.reshape((28, 28)), cmap=cmap, clim=clim)
    
    if len(title) > 0:
        plt.title(title)
        
    plt.axis('off')
    
    if colorbar:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.05)
        plt.colorbar(im, cax=cax)

In [ ]:
# More graphs

num_images_per_row  = 10
num_labels          = 10

plt.figure(figsize=(20, 20))
counter             = 1
for i in range(num_labels):
    for l in range(num_images_per_row):
        plt.subplot(num_labels, num_images_per_row, counter)

        all_images_belonging_to_class_l = X[y==l,:]
        one_images_belonging_to_class_l = all_images_belonging_to_class_l[i]
        
        show_image(one_images_belonging_to_class_l)  
        counter += 1



# 3. Explain what a training set is, what a test set is and why we as data scientists *always* split data into test/train before doing any modelling. What do we want to avoid?

A training set is a set of data that closely approximates the actual conditions of the sample you wish to study, without infringing on the specific conditions of the elements which you wish to test.  While we could both train and test on the same large set, the results would be suspiciously accurate, as the model would know the precise conditions for the testing set having encountered it in training, and the result would be falsely indicative of the true ability of the classifier.

Therefore, data scientists always split data into test/train and take great care in this, to avoid invalidating their own results with suspiciously high accuracies by leaks between the sets. 

In [ ]:
# No reason to change this, particularly.  It's a sensible split between test and train,
# and trying to prune the priors won't solve our problem

N = len(X)
N_train = int(0.8*N)
N_test = N-N_train

# set random seed:
np.random.seed(0) # don't change this :-)

# create a random permutation for splitting into training and test
randperm = np.random.permutation(N)

# split into training and test
train_idx = randperm[:N_train]
test_idx = randperm[N_train:]
Xtrain, Xtest = X[train_idx, :], X[test_idx, :]
ytrain, ytest = y[train_idx], y[test_idx]

print('Total number of images:\t\t%d' % N)
print('Number of training images:\t%d' % N_train)
print('Number of test images:\t\t%d' % N_test)
print(ytrain)

# 4. Implement/change the code to handle all digits. 
### Fitting the Naı̈ve-Bayes model to training set: the prior

In [ ]:
# count the number of zeros and ones, AND twos, threes, etc.


possible_numbers_vector = np.arange(10)
count_vector= np.arange(10)

for i in count_vector:
    count_vector[i] = np.sum(ytrain == i)

prob_of_numbers_vector = count_vector/ N_train #this vector contains the probability of each number to occur

# Prints all the priors, rounded.  Dataset overtrains to 1's, undertrains 5's, but look for 
# the raw vector and see if there's anything under - repped maybe?

# np.set_printoptions(threshold=np.inf) turn on for debugging
print(prob_of_numbers_vector)

for i in possible_numbers_vector:
    print('Prior probability (Y = %d)  = %d/%d = %3.2f' % (possible_numbers_vector[i], count_vector[i], N_train, prob_of_numbers_vector[i]))



The code beneath is taken from Exercise 1 and it only handles digits 0 and 1. Change the code to handle all digits.

### Fitting the Naı̈ve-Bayes model to training set: the likelihood

In [ ]:
### split data base on its label value and put them in a matrix 
# fit model for zeros and ones separately

p_numbers_vector = []
for k in range(10):
    # fit model for zeros and ones separately
    Xtrain_num = Xtrain[ytrain == k, :]
    # p(X_i = 1| Y = k) 
    p_numbers_vector.append(np.mean(Xtrain_num, axis=0))
p_digits = p_numbers_vector



In [ ]:
# This code should be sufficient, i.e. don't change this.
def log_likelihood(x_new, p_digit):
    pixel_log_lik = x_new*np.log(p_digit + 1e-16) + (1-x_new)*np.log(1-p_digit)
    return np.sum(pixel_log_lik)
    
image_idxs_to_be_classified = [0,10,510,810]
for image_idx in image_idxs_to_be_classified:
    x_new = Xtrain[image_idx]
    print('Image idx: %d (label=%d)' % (image_idx, ytrain[image_idx]))
    
    for i in range(10):
        print('p(x_new | Y=%d): %3.2e' % (i ,np.exp(log_likelihood(x_new, p_digits[i]))))
    print('\n')

# Continue from this part, greetings Pat 

### Implementing Bayes's theorem

In [ ]:
# Change the function to handle all digits (you may use for loop instead repeating the same code)

# the issue is to do with classification
# we have a valid array of all the posteriors generated but we are choosing the wrong one
# there are different reasons for this but it is no coincidence we are picking the same
# numbers as the overrepresented and underrepresented sets in the above

# we need to find a way for the system to choose the highest posterior
# I am assuming the issue is not with the actual classifier itself since that's
# as simple as it gets

def compute_posterior_prob(x_new):
    # compute log likelihood
    log_lik_zeros = log_likelihood(x_new, prob_of_numbers_vector[0])
    log_lik_ones = log_likelihood(x_new, prob_of_numbers_vector[1])
    log_lik_twos = log_likelihood(x_new, prob_of_numbers_vector[2])
    log_lik_threes = log_likelihood(x_new, prob_of_numbers_vector[3])
    log_lik_fours = log_likelihood(x_new, prob_of_numbers_vector[4])
    log_lik_fives = log_likelihood(x_new, prob_of_numbers_vector[5])
    log_lik_sixes = log_likelihood(x_new, prob_of_numbers_vector[6])
    log_lik_sevens = log_likelihood(x_new, prob_of_numbers_vector[7])
    log_lik_eights = log_likelihood(x_new, prob_of_numbers_vector[8])
    log_lik_nines = log_likelihood(x_new, prob_of_numbers_vector[9])
    # extend code here

    # exponentiate
    lik_zeros = np.exp(log_lik_zeros)
    lik_ones = np.exp(log_lik_ones)
    lik_twos = np.exp(log_lik_twos)
    lik_threes = np.exp(log_lik_threes)
    lik_fours = np.exp(log_lik_fours)
    lik_fives = np.exp(log_lik_fives)
    lik_sixes = np.exp(log_lik_sixes)
    lik_sevens = np.exp(log_lik_sevens)
    lik_eights = np.exp(log_lik_eights)
    lik_nines = np.exp(log_lik_nines)
    # extend code here

    # implement eq. (4)
    term_zeros = lik_zeros*prob_of_numbers_vector[0]
    term_ones = lik_ones*prob_of_numbers_vector[1]
    term_twos = lik_twos*prob_of_numbers_vector[2]
    term_threes = lik_threes*prob_of_numbers_vector[3]
    term_fours = lik_fours*prob_of_numbers_vector[4]
    term_fives = lik_fives*prob_of_numbers_vector[5]
    term_sixes = lik_sixes*prob_of_numbers_vector[6]
    term_sevens = lik_sevens*prob_of_numbers_vector[7]
    term_eights = lik_eights*prob_of_numbers_vector[8]
    term_nines = lik_nines*prob_of_numbers_vector[9]
    # extend code here
    evidence = term_zeros+term_ones+term_twos+term_threes+term_fours+term_fives+term_sixes+term_sevens+term_eights+term_nines
    
    # insert your code here to return a vector of length 10 containing the posterior probability
    # of belonging to each class:
    post_prob_zero = term_zeros/evidence    # change this code
    post_prob_one = term_ones/evidence     # change this code
    post_prob_two = term_twos/evidence     # change this code
    post_prob_three = term_threes/evidence   # change this code
    post_prob_four = term_fours/evidence    # change this code
    post_prob_five = term_fives/evidence    # change this code
    post_prob_six = term_sixes/evidence     # change this code
    post_prob_seven = term_sevens/evidence   # change this code
    post_prob_eight = term_eights/evidence   # change this code
    post_prob_nine = term_nines/evidence    # change this code

    # Collect all probabilities into a vector
    posterior = np.array([post_prob_zero,\
                          post_prob_one,\
                          post_prob_two,\
                          post_prob_three,\
                          post_prob_four,\
                          post_prob_five,\
                          post_prob_six,\
                          post_prob_seven,\
                          post_prob_eight,\
                          post_prob_nine]) 
    return posterior

### A simple classification rule: take the class with largest posterior probability

In [ ]:
def classify(x):
    posterior = compute_posterior_prob(x)
    predicted_label = np.argmax(posterior)
    return predicted_label
    

### Change this code to classify some images

In [ ]:
# Classifies the first 10 images in the test set
for i in range(100):
    f = plt.figure()
    
    # this set of classification rules makes little sense to me
    # but apparently they worked previously
    # there should be no issue with Classify but
    # we are still undoubtedly picking the wrong one
    
    # compute posterior probabilities
    posterior = compute_posterior_prob(Xtest[i, :])
    
    # get true label and predicted label
    true_label = ytest[i]
    predicted_label = classify(Xtest[i, :])
    
    # show image 
    show_image(Xtest[i, :])
    
    # Print result
    #print('p(Y|x) = (%2.1f)' % posterior)
    print(posterior)
    print("True label:",true_label)
    print("Predicted label:",predicted_label)
    


### Let's compute the training and test errors

In [ ]:
ytrain_hat = np.array([classify(x) for x in Xtrain])
ytest_hat = np.array([classify(x) for x in Xtest])

In [ ]:
mean_train_acc = np.mean(ytrain_hat == ytrain)
mean_test_acc = np.mean(ytest_hat == ytest)
print('Training accuracy:\t%4.3f' % mean_train_acc)
print('Test accuracy:\t\t%4.3f' % mean_test_acc)

# 5. Compute the confusion matrix and explain what it shows.

In [ ]:
from sklearn.metrics import confusion_matrix
labels = list(range(10))
pred = np.array([classify(x) for x in Xtest])
cm = confusion_matrix(y_true=ytest, y_pred=pred, labels=labels,normalize="true")
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
fig.colorbar(cax)
ax.set_xticks(labels)
ax.set_yticks(labels)
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)
plt.tight_layout()
plt.show()

Write your explanation here

# 6. Error analysis: find images that are misclassified by the system.  Are there common characteristics among the images that are misclassified?
It's down to "ornamentation" from what I can determine after going over the array of posteriors but I can't prove it without fixing the system